In [126]:
import keras
from keras.models import Sequential
from keras.layers import Dense, Activation, Dropout, Flatten
from keras.layers import Conv2D
from keras.layers import MaxPooling2D
from keras.optimizers import SGD
from keras.preprocessing.image import ImageDataGenerator
import numpy as np

In [106]:
input_shape = (48, 48, 1)
classes = 3

model = Sequential([
    # block 1
    Conv2D(64, (3, 3), input_shape=input_shape, padding='same', activation='relu'),
    Conv2D(64, (3, 3), activation='relu', padding='same'),
    MaxPooling2D(pool_size=(2, 2), strides=(2, 2)),
    
    # block 2
    Conv2D(128, (3, 3), activation='relu', padding='same'),
    Conv2D(128, (3, 3), activation='relu', padding='same',),
    MaxPooling2D(pool_size=(2, 2), strides=(2, 2)),
    
    # block 3
    Conv2D(256, (3, 3), activation='relu', padding='same',),
    Conv2D(256, (3, 3), activation='relu', padding='same',),
    Conv2D(256, (3, 3), activation='relu', padding='same',),
    MaxPooling2D(pool_size=(2, 2), strides=(2, 2)),
    
    # block 4
    Conv2D(512, (3, 3), activation='relu', padding='same',),
    Conv2D(512, (3, 3), activation='relu', padding='same',),
    Conv2D(512, (3, 3), activation='relu', padding='same',),
    MaxPooling2D(pool_size=(2, 2), strides=(2, 2)),
    
    # block 5
#     Conv2D(512, (3, 3), activation='relu', padding='same',),
#     Conv2D(512, (3, 3), activation='relu', padding='same',),
#     Conv2D(512, (3, 3), activation='relu', padding='same',),
#     MaxPooling2D(pool_size=(2, 2), strides=(2, 2)),
    
    # top 
    Flatten(),
    Dense(4096, activation='relu'),
    Dense(4096, activation='relu'),
    Dense(classes, activation='softmax')
])

model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_109 (Conv2D)          (None, 48, 48, 64)        640       
_________________________________________________________________
conv2d_110 (Conv2D)          (None, 48, 48, 64)        36928     
_________________________________________________________________
max_pooling2d_43 (MaxPooling (None, 24, 24, 64)        0         
_________________________________________________________________
conv2d_111 (Conv2D)          (None, 24, 24, 128)       73856     
_________________________________________________________________
conv2d_112 (Conv2D)          (None, 24, 24, 128)       147584    
_________________________________________________________________
max_pooling2d_44 (MaxPooling (None, 12, 12, 128)       0         
_________________________________________________________________
conv2d_113 (Conv2D)          (None, 12, 12, 256)       295168    
__________

In [107]:
import os
os.listdir('../input')

['test_data.csv',
 'train_target.csv',
 'sample-submission.csv',
 'train_data.csv']

In [127]:
import pandas as pd

# data directories
train_dir = '../input'
train_csv = '/train_data.csv'

# read csv
df = pd.read_csv(train_dir+train_csv, header=None)

# array of 16.2k 48x48 image arrays
nb_total_train = 16175
img_wh = 48

# split into train / test manually
nb_training_samples = 14175
nb_testing_samples = 2000

train_data = np.zeros((nb_training_samples, img_wh, img_wh, 1))
test_data = np.zeros((nb_testing_samples, img_wh, img_wh, 1))

# iterate over rows and reshape as 2304x1 array to 48x48 array
for i, row in df.iterrows():
    img = np.asarray(row).reshape((48,48,1))
    if i < nb_training_samples:
        train_data[i] = img
    else:
        test_data[i-nb_training_samples] = img

In [109]:
print(train_data.shape)
print(test_data.shape)

(14175, 48, 48, 1)
(2000, 48, 48, 1)


In [110]:
row.shape

(2304,)

In [133]:
df.head()

,0,1,2,3,4,5,6,7,8,9,...,2294,2295,2296,2297,2298,2299,2300,2301,2302,2303
0,146,146,145,145,145,142,140,114,126,128,...,103,103,93,87,73,65,65,76,85,81
1,9,12,34,72,98,110,113,116,126,134,...,141,121,131,118,93,74,31,7,4,9
2,255,129,62,59,45,31,29,26,32,23,...,122,130,138,113,43,26,16,12,14,16
3,217,191,222,176,150,201,157,72,59,74,...,223,229,114,70,111,95,71,109,159,191
4,19,11,4,37,62,78,98,113,121,126,...,152,149,156,161,160,158,160,177,187,181


In [129]:
train_data[0]

array([[[146.],
        [146.],
        [145.],
        ...,
        [ 93.],
        [ 78.],
        [ 78.]],

       [[143.],
        [143.],
        [144.],
        ...,
        [129.],
        [ 83.],
        [ 80.]],

       [[128.],
        [128.],
        [129.],
        ...,
        [152.],
        [100.],
        [ 70.]],

       ...,

       [[ 92.],
        [101.],
        [ 93.],
        ...,
        [ 67.],
        [ 78.],
        [ 85.]],

       [[ 83.],
        [ 99.],
        [100.],
        ...,
        [ 69.],
        [ 82.],
        [ 82.]],

       [[ 82.],
        [ 97.],
        [104.],
        ...,
        [ 76.],
        [ 85.],
        [ 81.]]])

In [130]:
target_csv = '/train_target.csv'

target_df = pd.read_csv(train_dir+target_csv, header=None)

# split into train / test manually
train_labels = np.zeros((nb_training_samples,1))
test_labels = np.zeros((nb_testing_samples,1))

for i, row in target_df.iterrows():
    if i < nb_training_samples:
        train_labels[i] = row
    else:
        test_labels[i-nb_training_samples] = row

# convert to keras friendy format
train_labels = keras.utils.to_categorical(train_labels, num_classes=classes)
test_labels = keras.utils.to_categorical(test_labels, num_classes=classes)

In [114]:
print(train_labels.shape)
print(test_labels.shape)

(14175, 3)
(2000, 3)


In [115]:
# Generate dummy data
x_train = np.random.random((100, 100, 100, 3))
y_train = keras.utils.to_categorical(np.random.randint(10, size=(100, 1)), num_classes=10)
x_test = np.random.random((20, 100, 100, 3))
y_test = keras.utils.to_categorical(np.random.randint(10, size=(20, 1)), num_classes=10)
print(x_train.shape)
print(y_train.shape)

(100, 100, 100, 3)
(100, 10)


In [134]:
image_gen = ImageDataGenerator(
    width_shift_range=0.15,
    height_shift_range=0.15,
    rotation_range=20,
    shear_range=0.01,
    zoom_range=[.9, 1.2],
    horizontal_flip=True,
    vertical_flip=False,
    fill_mode='reflect'
)

In [135]:
sgd = SGD(lr=0.01, decay=1e-6, momentum=0.9, nesterov=True)
model.compile(loss='categorical_crossentropy', optimizer=sgd)

model.fit(train_data, train_labels, batch_size=32, epochs=15)

Epoch 1/15
14175/14175 [==============================] - 43s 3ms/step - loss: 0.7460
Epoch 2/15
14175/14175 [==============================] - 41s 3ms/step - loss: 0.7433
Epoch 3/15
14175/14175 [==============================] - 41s 3ms/step - loss: 0.6700
Epoch 4/15
14175/14175 [==============================] - 41s 3ms/step - loss: 0.6720
Epoch 5/15
14175/14175 [==============================] - 41s 3ms/step - loss: 0.6876
Epoch 6/15
14175/14175 [==============================] - 41s 3ms/step - loss: 0.7120
Epoch 7/15
14175/14175 [==============================] - 41s 3ms/step - loss: 0.6828
Epoch 8/15
14175/14175 [==============================] - 41s 3ms/step - loss: 0.7452
Epoch 9/15
14175/14175 [==============================] - 41s 3ms/step - loss: 0.6841
Epoch 10/15
14175/14175 [==============================] - 41s 3ms/step - loss: 0.7312
Epoch 11/15
14175/14175 [==============================] - 41s 3ms/step - loss: 0.7348
Epoch 12/15
14175/14175 [===========================

In [136]:
score = model.evaluate(test_data, test_labels, batch_size=32)
score

2000/2000 [==============================] - 2s 1ms/step


0.8447884330749512

In [137]:
model.save_weights('lr.01_decay1e-6_mom.9_batch32_epocs15a.h5')